In [1]:
from skimage.io import imread
import numpy as np
import fnmatch
import os
import re
import random

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.utils import Sequence, to_categorical

import sys
if "E:\\git\\keras-resnet" not in sys.path:
    sys.path.append("E:\\git\\keras-resnet")
    print(sys.path)
    
import resnet



Using TensorFlow backend.


['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet']


In [2]:
#TODO check shuffle
class CustomDataGenerator(Sequence):

    def __init__(self, directory, batch_size, classes):
        self.directory = directory
        self.batch_size = batch_size
        self.classes = classes
        
        #create filenames, labels
        dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
        print(dirs)
        self.files = []
        for d in dirs:
            fullpath_dir = os.path.join(directory,d)
            new_elements = [(f,d) for f in os.listdir(fullpath_dir)]
            self.files.extend(new_elements)
        
        # shuffle
        random.shuffle(self.files)
        print(len(self.files))
    
    
    def getLabels(self, offsetStr, viewStr): 
        offset = 0
        view = 0
        for i, e in enumerate(self.classes[0]):
            if e == offsetStr:
                offset = i
                break
            
        for i, e in enumerate(self.classes[1]):
            if e == viewStr:
                view = i
                break
            
        return offset, view
    
    
    def __len__(self):
        return int(np.ceil(len(self.files) / float(self.batch_size)))

    def on_epoch_end(self):
        'Shuffle for the next epoch'
        random.shuffle(self.files)
    
    def __getitem__(self, idx):
        batch_x = []
        batch_y1 = []
        batch_y2 = []
        batch = self.files[idx * self.batch_size : (idx + 1) * self.batch_size]
        #print(self.directory)
        #print ('files from getitem(): ', batch)
        p = re.compile('(left|center|right)Offset_(left|straight|right)View')
        
        for (file, d) in batch:
            
            finds = p.findall(d)
            #TODO assert only finds has only 1 element [(_,_)]
            (offsetStr, viewStr) =  finds[0]
            (offset, view) = self.getLabels(offsetStr, viewStr)
            
            folder = os.path.join(self.directory, d)
            rgba = imread(os.path.join(folder, file))
            rgb = rgba[:,:,:3]
            
            input_array = rgb / 255.0
            
            batch_x.append(input_array)
            batch_y1.append(offset)
            batch_y2.append(view)
         
        return np.array(batch_x), [np.array(to_categorical(batch_y1, 3)), np.array(to_categorical(batch_y2, 3))]


In [3]:
#sanity test CustomDataGenerator
# directory = directory = "E:\\UAV_drone\\data\\training\\lateral_and_view_normal\\train"
# classes = [['left', 'center', 'right'], ['left', 'straight', 'right']]
# datagen = CustomDataGenerator(directory, 5, classes)
# datagen.__getitem__(0)


In [4]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'C:\\Users\\hoang\\WorkingSpace\\TraningModels\\data_training\\lateral_and_view_normal\\train'
val_data_dir = 'C:\\Users\\hoang\\WorkingSpace\\TraningModels\\data_training\\lateral_and_view_normal\\val'
nb_train_samples = 44293
nb_validation_samples = 11620
epochs = 50
batch_size = 16
img_channels = 3
nb_classes = 3

if K.image_data_format() == 'channels_first':
    input_shape = (img_channels, img_width, img_height)
else:
    input_shape = (img_width, img_height, img_channels)

model = resnet.ResnetBuilder.build_resnet_18_uav((img_channels, img_width, img_height), nb_classes, multi_output=2)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
classes = [['left', 'center', 'right'], ['left', 'straight', 'right']]
train_datagen = CustomDataGenerator(train_data_dir, batch_size, classes)

# this is the augmentation configuration we will use for testing:
val_datagen = CustomDataGenerator(val_data_dir, batch_size, classes)


# checkpoint
current_directory = os.getcwd()
checkpoint_dir = os.path.join(current_directory, 'checkpoint')
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

file="model-uav-view_offset-normal-{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)


['centerOffset_leftView', 'centerOffset_rightView', 'centerOffset_straightView', 'leftOffset_leftView', 'leftOffset_rightView', 'leftOffset_straightView', 'rightOffset_leftView', 'rightOffset_rightView', 'rightOffset_straightView']
44293
['centerOffset_leftView', 'centerOffset_rightView', 'centerOffset_straightView', 'leftOffset_leftView', 'leftOffset_rightView', 'leftOffset_straightView', 'rightOffset_leftView', 'rightOffset_rightView', 'rightOffset_straightView']
11620


In [5]:
model.fit_generator(
    train_datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_datagen,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint])

Epoch 1/50
2768/2768 [==============================] - 1554s 561ms/step - loss: 1.5586 - dense_1_loss: 0.7732 - dense_2_loss: 0.4521 - dense_1_acc: 0.6047 - dense_2_acc: 0.7917 - val_loss: 1.1604 - val_dense_1_loss: 0.4821 - val_dense_2_loss: 0.4390 - val_dense_1_acc: 0.8068 - val_dense_2_acc: 0.7806

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-01.hdf5
Epoch 2/50
2768/2768 [==============================] - 1421s 513ms/step - loss: 0.7285 - dense_1_loss: 0.3914 - dense_2_loss: 0.1245 - dense_1_acc: 0.8466 - dense_2_acc: 0.9586 - val_loss: 1.0148 - val_dense_1_loss: 0.6083 - val_dense_2_loss: 0.2157 - val_dense_1_acc: 0.6692 - val_dense_2_acc: 0.9261

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-02.hdf5
Epoch 3/50
2768/2768 [==============================] - 1293s 467ms/step - loss: 0.5669 - dense_1_loss: 0.2896 - dense_2_loss: 0.0996 - dense_1_acc:

2768/2768 [==============================] - 1451s 524ms/step - loss: 0.1975 - dense_1_loss: 0.0621 - dense_2_loss: 0.0389 - dense_1_acc: 0.9766 - dense_2_acc: 0.9866 - val_loss: 0.5640 - val_dense_1_loss: 0.3271 - val_dense_2_loss: 0.1408 - val_dense_1_acc: 0.9127 - val_dense_2_acc: 0.9470

Epoch 00020: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-20.hdf5
Epoch 21/50
2768/2768 [==============================] - 1466s 530ms/step - loss: 0.1921 - dense_1_loss: 0.0595 - dense_2_loss: 0.0370 - dense_1_acc: 0.9787 - dense_2_acc: 0.9868 - val_loss: 0.5986 - val_dense_1_loss: 0.3681 - val_dense_2_loss: 0.1357 - val_dense_1_acc: 0.8522 - val_dense_2_acc: 0.9492

Epoch 00021: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-21.hdf5
Epoch 22/50
2768/2768 [==============================] - 1344s 485ms/step - loss: 0.1880 - dense_1_loss: 0.0573 - dense_2_loss: 0.0357 - dense_1_acc: 0.9785 -

2768/2768 [==============================] - 1415s 511ms/step - loss: 0.1550 - dense_1_loss: 0.0425 - dense_2_loss: 0.0240 - dense_1_acc: 0.9848 - dense_2_acc: 0.9917 - val_loss: 2.3766 - val_dense_1_loss: 2.0446 - val_dense_2_loss: 0.2440 - val_dense_1_acc: 0.6088 - val_dense_2_acc: 0.9244

Epoch 00039: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-39.hdf5
Epoch 40/50
2768/2768 [==============================] - 1295s 468ms/step - loss: 0.1560 - dense_1_loss: 0.0431 - dense_2_loss: 0.0246 - dense_1_acc: 0.9851 - dense_2_acc: 0.9913 - val_loss: 0.5589 - val_dense_1_loss: 0.2854 - val_dense_2_loss: 0.1842 - val_dense_1_acc: 0.9054 - val_dense_2_acc: 0.9336

Epoch 00040: saving model to C:\Users\hoang\WorkingSpace\TraningModels\keras-resnet\checkpoint\model-uav-vertical-40.hdf5
Epoch 41/50
2768/2768 [==============================] - 1413s 510ms/step - loss: 0.1518 - dense_1_loss: 0.0394 - dense_2_loss: 0.0245 - dense_1_acc: 0.9854 -